# 生成 AI による記事アイデアの作成

この Notebook では、生成 AI に記事のネタだしから調査、記事作成までを自動で実行する Agent を作成します。

PV 数の多い記事を分析して得た PV 数が高くなりそうな記事のネタの仮説を元に、Web 検索を利用し情報を集めて記事アイデアを提案するという設計にします。

## Agent の作成

まずは、 `langchain` を使用した ReACT エージェントを作成します。

In [2]:
!pip install "langchain==0.0.309" langchainhub -qU
!pip install "duckduckgo-search>=3.9.5" -qU

In [3]:
import langchain
from langchain.llms.bedrock import Bedrock
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools.render import render_text_description
from langchain.agents import Tool, AgentExecutor
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain import hub
import boto3

# langchain.verbose = True

# Initialize Model
bedrock_client = boto3.client("bedrock-runtime")
llm = Bedrock(
    model_id="anthropic.claude-instant-v1",
    client=bedrock_client,
    model_kwargs={ 'max_tokens_to_sample': 1024 }
)

# Initialize Tool
wrapper = DuckDuckGoSearchAPIWrapper(region="jp-jp", safesearch="strict")
search = DuckDuckGoSearchRun(api_wrapper=wrapper)
tools = [
    Tool(
        name="duckduckgo-search",
        func=search.run,
        description="日本語もしくは英語でウェブ検索が可能",
    )
]

デフォルトの `langchain` の Agent のプロンプトは GPT 用に作成されているため、`Claude` の[プロンプトベストプラクティス](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)を反映したプロンプトでテンプレートを上書きします。

In [4]:
# Define Prompt
prompt = hub.pull("hwchase17/react")
tool_names = ", ".join([t.name for t in tools])
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=tool_names,
)
print("元のプロンプト: ", prompt.template)
print("---")
prompt.template = """

Human: 以下の instruction で与えられた指示に従ってください。以下のツールを利用することが可能です。

<ツール>
{tools}
</ツール>

以下のフォーマットを使用してください:

<output-format>
Thought: 指示に答えるために何をするべきか検討
Action: 実行するアクション。必ず {tool_names} から選択する。
Action Input: Action への入力
Observation: アクションの結果
... (Thought/Action/Action Input/Observation を N 回繰り返す)
Thought: 答えるのに必要な情報が揃いました
Final Answer: instruction に対する出力のみ <output><output> で囲んで出力
</output-format>

それでは以下から開始してください。

<instruction>
Instruction: {input}
</instruction>

Assistant:
Thought:{agent_scratchpad}
"""
print("Claude に最適化されたプロンプト: ", prompt.template)

# Stop Generation on stop token (passed to Bedrock)
llm_with_stop = llm.bind(stop=["\nObservation"])

# Create Agent
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps'])
} | prompt | llm_with_stop | ReActSingleInputOutputParser()

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

元のプロンプト:  Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
---
Claude に最適化されたプロンプト:  

Human: 以下の instruction で与えられた指示に従ってください。以下のツールを利用することが可能です。

<ツール>
{tools}
</ツール>

以下のフォーマットを使用してください:

<output-format>
Thought: 指示に答えるために何をするべきか検討
Action: 実行するアクション。必ず {tool_names} から選択する。
Action Input: Action への入力
Observation: アクションの結果
... (Thought/Action/Action Input/Observation を N 回繰り返す)
Thought: 答えるのに必要な情報が揃いました
Final Answer: instruction に対する出力のみ <output><output> で囲んで出力
</ou

## 出力のハイライト

著作権の配慮および Hallucination の検知のために、出力のうち、検索で取得したデータをハイライトして表示します。

In [5]:
from IPython.display import display, HTML

colors = [
    "#FF0000",
    "#00FF00",
    "#FFFF00",
    "#FF00FF",
    "#00FFFF",
    "#CC0000",
    "#00CC00",
    "#CCCC00",
    "#CC00CC",
    "#00CCCC",
    "#990000",
    "#009900",
    "#999900",
    "#990099",
    "#009999",
]


def create_ngram(string, n=2):
    delimiter = ['「', '」', '…', '　']
    if n == 1:
        return list(string)
    elif n == 2:
        double = list(zip(string[:-1], string[1:]))
        double = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter))), double)
        return list(double)
    elif n == 3:
        triple = list(zip(string[:-2], string[1:-1], string[2:]))
        triple = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter) or (x[2] in delimiter))), triple)
        return list(triple)
    return []


def create_ngrams(string):
    return create_ngram(string, 1)


def highlight(string, source):
    return f'<span class="c{source}">{"".join(string)}</span>'


def process_result(result):
    output = result["output"].replace("<output>", "").replace("</output>", "")
    search_results = "".join([x[1].replace("...", "\n\n") for x in result["intermediate_steps"]]).split("\n\n")

    # 一文字ごとにトークンとして分割
    output_tokens = list(output)
    search_tokens = [list(search_result) for search_result in search_results]
    
    # ngram: 1
    ngram_dict = {}
    for idx, search_token in enumerate(search_tokens):
        ngrams = create_ngrams(search_token)
        for ngram in ngrams:
            if not ngram in ngram_dict:
                ngram_dict[ngram] = set()
            ngram_dict[ngram] = ngram_dict[ngram] | set([idx])
    
    # Match longest consecutive ngrams from single source and highlight
    html = ""
    buffer = ()
    buffer_source = set()
    used_source = set()
    for token in output_tokens:
        if not token in ngram_dict:
            if buffer:
                source = list(buffer_source)[0]
                html += highlight("".join(buffer), source)
                buffer = ()
                used_source |= set([source])
                buffer_source = set()
            html += token
        else:
            current_source = ngram_dict[token]
            if not buffer:
                buffer = [token]
                buffer_source = current_source
            else:
                # Check for overlap 
                if buffer_source & current_source:
                    buffer.append(token)
                    buffer_source = buffer_source & current_source
                # No overlap, flush buffer
                else:
                    source = list(buffer_source)[0]
                    html += highlight("".join(buffer), source)
                    buffer = [token]
                    used_source |= set([source])
                    buffer_source = current_source
    if buffer:
        html += highlight("".join(buffer), list(buffer_source)[0])
    
    html = html.replace("\n", "<br/>")
    style = "<style>" + "\n".join([
        f".c{source} " + "{ color: " + colors[idx % len(colors)] + " !important; }" for idx, source in enumerate(used_source)
    ]) + "</style>"

    return style + "<div>" + html + "</div>"


# processed_result = process_result(result)
# print(processed_result)
# HTML(processed_result)

## Agent の検証

In [6]:
result = agent_executor.invoke({
    "input": "あなたは敏腕記者です。今話題の日本の芸能人で話題になりそうなものを一つピックアップし深掘りして関連情報などを調査し1500文字程度のニュース記事を作成してください。"
})
print("--- 出力 ---")
HTML(process_result(result))



> Entering new AgentExecutor chain...
日本の芸能人で現在話題になっている人物がどれかを調べる必要があります。

Action: duckduckgo-search
Action Input: "現在日本で話題になっている芸能人"
この記事では、松嶋菜々子さんの 若い頃から現在2... 小田茜 2023年女優復帰された小田茜さん。 かつての国民的美少女が結婚、離婚を経験し、現在はヨガインストラクターをやりながら女優復帰をされています。 タレントの熊田曜子が、12日までに自身のインスタグラムを更新。 動画をアップし「#始めて2年」が経過した、ポールダンスの腕前を披露した。 2023-11-12 10:40 ｜SNS発｜ 6 松田美由紀、夫・優作の「ベスパ」公開 『探偵物語』で愛用"伝説の白バイク"は現存していた「我が家に何十年も地下室にあった」... rizinに出場している人気格闘家の平本蓮（ひらもと・れん）さん。 恋愛リアリティ番組に出演したり、同じく人気格闘家の朝倉未来さんとのsnsトラブルなど、何かと話題になっている平本蓮さん。 2023年4月には文春で熱愛が報じられていたようですが、現在 ... 以上が2023年下半期にブレイクすることが予想されるタレント5選でした。. 彼らの今後の動向に今から見逃せません!. 今回ご紹介した方々に皆さんも是非注目してみてください。. お問い 合わせはこちらから. Tel：03-6712-5946. 記事：鹿野 七々華 (Youth Clip編集 ... 最新の芸能 ニュースや特集をいち早くお届け!最新情報を発信する総合トレンドメディア「oricon news」では、話題の芸能 のニュースや特集を画像 ...
平本蓮さんは動向が注目されており、格闘技と芸能の世界で活動されていることから、深掘りできそうな人物だと思います。

Action: duckduckgo-search 
Action Input: "平本蓮"
2023年4月29日（土）国立代々木競技場 第一体育館にて開催された「FEDELTA presents RIZIN LANDMARK 5 in YOYOGI」試合フル映像 NEXT EVENT「RIZIN.42」in 有明 ... RIZINとベラトールの5vs5対抗戦をは

In [7]:
result = agent_executor.invoke({
    "input": "あなたは敏腕経済記者です。直近の日本の企業ニュースを一つピックアップし深掘り調査し深掘りして関連情報などを調査し1500文字程度のニュース記事を作成してください。"
})
print("--- 出力 ---")
HTML(process_result(result))



> Entering new AgentExecutor chain...
Instructionにあるとおり日本の最近の企業ニュースを探す必要がある

Action: duckduckgo-search
Action Input: "recent japanese corporate news"
TOKYO (Reuters) — Visitors to Japan exceeded pre-pandemic levels in October, official data showed on Wednesday, marking a full recovery in arrivals for the first time since the relaxation of border controls last year. The number of foreign visitors for business and leisure rose to 2.52 million last month from 2.18 million in September, data from the Japan National Tourism Organization (JNTO ... November 14, 2023. Business Japan's Denso to target $8 bln in electrification revenue by FY 2025. Japanese automotive supplier Denso aims to increase its revenue from electrification to 1.2 ... Yamanashi wineries preparing to ship 'nouveau' products. The latest business and technology news and trends in Asia and beyond. Japan's public broadcaster offers articles, videos, and more. Advertisement · Scroll to continue. China detained the executive, 

In [15]:
result = agent_executor.invoke({
    "input": "あなたは敏腕IT記者です。AWS の2023年10月のアナウンスから話題になりそうなものを一つピックアップし深掘りして関連情報などを調査し1500文字程度のニュース記事を作成してください。"
})
print("--- 出力 ---")
HTML(process_result(result))



> Entering new AgentExecutor chain...
AWSの10月のアナウンスから話題になりそうなものをピックアップする必要がある

Action:
duckduckgo-search

Action Input:
AWS 2023 October announcement
Stay updated with the latest offerings from AWS Training and Certification in the October 2023 edition of the blog. Discover new courses, learning paths, certifications, and resources to help you advance your cloud skills and career goals. Whether you are a beginner or an expert, you can find something that suits your needs and interests. 10/23/2023, 12:00:00 AM ~ 10/24/2023, 12:00:00 AM (UTC) Recent Announcements# AWS Marketplace announces enhanced private offer user experience for sellers. Today, AWS Marketplace announces the general availability of an enhanced private offer user experience to improve ease of use across different product types and pricing models. With the announcement of several powerful new offerings to accelerate AI innovation, AWS further reinforces its position as one of the leading platforms to host generation AI

## 記事アイデアの作成

Agent がインターネットから必要な情報を検索して記事を書く検証ができたため、次は PV 数が高い記事を分析した結果をもとに、ヒットする記事の仮説を立てて必要な情報を収集し記事の草案を作成させます。

In [8]:
import pandas as pd
import os

output_path = "analysis.csv"
if os.path.exists(output_path):
    hits = pd.read_csv(output_path)
    for idx, row in hits.iterrows():
        print(row["analysis"])
        print("")

- クイズ形式の記事は読者の知識を試す面白さがある
- テレビ番組に関するクイズは、多くの人が親しみを感じる題材
- ジェパディの実際の問題を使っていることで、クイズの難易度と面白さが保証される
- 記事タイトルがクイズ形式で、読者の興味を引く
- 記事の概要で、クイズの形式と内容が明確に説明されている
- 同じ形式と題材のクイズ記事を定期的に提供することで、読者を獲得できる

- タイトルに「最も」「全米No.1」といった強調表現を用い、読者の興味を引く
- 高級ホテルという非日常的な体験に対する人々の憧れをテーマにする
- セレブリティが訪れるホテルという著名人との接点をアピールする
- 掲載ホテルの写真を多用し、非日常的な雰囲気を伝える 
- SNSでのシェア数をアピールすることで話題性を高める

- 「あらゆるタイプの人」「あらゆる予算」というキーワードがあるので、多くの人に関心のある普遍的なテーマだと思われる。
- 「驚くべき」「完璧な」といった言葉でクリック率を高めている。
- 誕生日やクリスマスなどの贈り物選びのシーズンに合わせたタイミングの良い記事。
- Reviewedという信頼できる情報源からの贈り物のおすすめ記事。
- 具体的な商品のおすすめがあるため、実際に購入に結びつきやすい。

- セレブリティの金銭トラブルは注目を集める話題だ。セレブの破産や巨額の報酬、資産など金銭に関するゴシップ記事を作ることでPVを獲得できる。
- 過去10年間の出来事をまとめたリスト形式は読みやすくPVを集めやすい。時代背景を踏まえた10年間のトレンドを分析した記事を作れば再現できる。
- 著名人のスキャンダルやゴシップは関心が高く、クリック率が良い。セレブの金銭問題を扱った記事を作ればPVを獲得できる。

- 有名人の軍歴に関する記事であるため、有名人の過去の秘話や意外な一面に関心がある読者を引きつけている
- 具体的な有名人の名前が記載されているため、その有名人のファンも記事に興味を持つ
- 軍隊経験は一般人にはなじみが薄い世界なので、著名人のそうした経験に興味が惹かれる
- 同様の著名人の過去のエピソードや意外な経歴を扱った記事を制作することで、PVを再現できる可能性がある

- アメリカの上院議員に関する記事であるため、アメリカの政治に関心のある読者を引き

In [10]:
for idx, row in hits[:3].iterrows():
    print("")
    print("##########################")
    print("--- PV 数の多い記事の要素 ---")
    print(row["analysis"])
    result = agent_executor.invoke({
        "input": f"""あなたは記者です。PV 数の多い記事の要素をもとに記事のアイデアを作成し、必要な情報を調査し記事の原稿としてまとめあげて報告してください。
<PV 数の多い記事の要素>
{row["analysis"]}
</PV 数の多い記事の要素>"""
    })
    print("--- 出力 ---")
    display(HTML(process_result(result)))


##########################
--- PV 数の多い記事の要素 ---
- クイズ形式の記事は読者の知識を試す面白さがある
- テレビ番組に関するクイズは、多くの人が親しみを感じる題材
- ジェパディの実際の問題を使っていることで、クイズの難易度と面白さが保証される
- 記事タイトルがクイズ形式で、読者の興味を引く
- 記事の概要で、クイズの形式と内容が明確に説明されている
- 同じ形式と題材のクイズ記事を定期的に提供することで、読者を獲得できる


> Entering new AgentExecutor chain...
抽象的な要素から具体的なクイズ記事のアイデアを導き出す必要がある

Action: duckduckgo-search
Action Input: ジェパディの問題を使ったテレビ番組クイズ記事の例
^米長寿クイズ番組『ジェパディ!』司会者 トレベックさん死去-nhk ^ 『IBM奇跡の"ワトソン"プロジェクト──人工知能はクイズ王の夢を見る』ハヤカワ・オンライン ^ なお、賭け金をゼロにすることも可能だが、当然、正解しても$1ももらえない。 ^ 2位以下は対戦で得た金額を得ることは出来 ... ジェパディ作成の手順 1. カテゴリーの設定 2. 問題の作成 3. ジェパディの完成 無料で遊べるオンラインジェパディの紹介 A. Bible Jeopardy B. Jeopardy Labsの公式サイト C. Jeopardy Rocks D. Classroom Jeopardy ジェパディを使った学習活動の例 英語学習 歴史学習 科学学習 ... アメリカの早押しクイズだと、クイズ番組の「ジェパディ!」のように問題文が読み切られてからボタンを押す形式と競技クイズの「クイズボウル」のように読み上げ途中でもボタンを押すことが可能な形式などがあります。 ただ、日本の早押しクイズの ... ^米長寿クイズ番組『ジェパディ!』司会者 トレベックさん死去-nhk ^ 『IBM奇跡の"ワトソン"プロジェクト──人工知能はクイズ王の夢を見る』ハヤカワ・オンライン ^ なお、賭け金をゼロにすることも可能だが、当然、正解しても$1ももらえない。 ^ 2位以下は対戦で得た金額を得ることは出来 ... ジャニーズ事務所の創業者である故ジャニー


##########################
--- PV 数の多い記事の要素 ---
- タイトルに「最も」「全米No.1」といった強調表現を用い、読者の興味を引く
- 高級ホテルという非日常的な体験に対する人々の憧れをテーマにする
- セレブリティが訪れるホテルという著名人との接点をアピールする
- 掲載ホテルの写真を多用し、非日常的な雰囲気を伝える 
- SNSでのシェア数をアピールすることで話題性を高める


> Entering new AgentExecutor chain...
 Instructionから提示されたPV数の多い記事の要素をもとに、記事のアイデアと必要な情調査内容を考える

<output-format>

Action: duckduckgo-search
Action Input: アメリカの高級ホテルアメリカ・ロサンゼルスの高級ホテルでぜいたくなホテルステイを! アメリカ・ロサンゼルスには5つ星ホテルや歴史ある老舗の高級ホテル、ラウンジやスパがあるラグジュアリーなホテルまで、さまざまな特徴がある高級ホテルがたくさん! 先日ロサンゼルスへ旅行した際に、渡米以来ずっと狙っていたホテルに宿泊してきました。. そのホテルの名は、ウォルドーフ・アストリア・ビバリーヒルズ（Waldorf Astoria Beverly Hills）!. そう、あのセレブの街『ビバリーヒルズ』にあるヒルトン系最高級 ... こちらでは10万円以上の最高級ホテルと6 ... ペニンシュラのアメリカ進出第一号店である「ザ ペニンシュラ ニューヨーク」。元のゴッサムホテルから換算すると開業から100年を超える歴史があり、ニューヨーク独特の世界観を感じられるホテル。 ... この記事では、アメリカ・ニューヨークでおすすめの高級ホテルを紹介します。おすすめの高級ホテル8つに加えて、高級ホテルを選ぶ際のポイントを紹介しているので、ニューヨークで高級ホテルを探している方はぜひご覧ください。 この記事では アメリカ大手ホテルチェーンのマリオットとヒルトンに100泊以上し、上級ステータスを獲得している 私が、様々な観点で比較してお伝えします。. 私の会員ステータスと宿泊数（記念すべき100泊）. 記事を書いている時点で駐在して1年半ほど ... ロサンゼルスとニューヨ


##########################
--- PV 数の多い記事の要素 ---
- 「あらゆるタイプの人」「あらゆる予算」というキーワードがあるので、多くの人に関心のある普遍的なテーマだと思われる。
- 「驚くべき」「完璧な」といった言葉でクリック率を高めている。
- 誕生日やクリスマスなどの贈り物選びのシーズンに合わせたタイミングの良い記事。
- Reviewedという信頼できる情報源からの贈り物のおすすめ記事。
- 具体的な商品のおすすめがあるため、実際に購入に結びつきやすい。


> Entering new AgentExecutor chain...
<instruction> から提示されたPVの高い記事の特徴から、新しい記事のアイデアを考える必要がある。
Action: duckduckgo-search
Action Input: "PVの高い記事の要素をもとにインスピレーションを得る"PV数を向上させる施策を行う前に、知っておきたい基礎知識が2つあります。 ここでは、それぞれ詳しく見ていきましょう。 PVが多くてもコンバージョン率が低いこともある 大前提として知ってもらいたいのが、PV数が多くてもコンバージョンに発展しないケースがあるということです。 これは、SEO対策をゴリゴリに行なったコンテンツで、ブラックハットに近い場合に多く見られる現象です。 このケースにおいては、検索順位で上位表示される可能性は高いけれど、ユーザーのニーズを満たせるコンテンツにはなっておらず、ページの滞在時間が短い傾向にあります。 PVのことだけを考えるとSEO対策を実施したくなりがちですが、あくまで最終目標はコンバージョンを増やし、商品を販売することです。 オウンドメディアで目標PV数を達成させるためには？というテーマは、多くの企業やブログ運営者にとって課題です。この記事ではオウンドメディアの魅力を引き出し、目標PV数を達成するためのヒント、改善策を紹介していきます。 実際のPV数と収益を公開. これは私が運営するアドセンスブログの実際のデータです。. ページビュー（PV）：14.4万 クリック数：1,000 CPC（クリック単価）：15円. 上記の場合、アドセンス収益額は、 1,000クリック × 15円 ＝15,000円 になります。. 広告の表示